In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import SGDClassifier

In [13]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [14]:
train = pd.read_csv("location_train.csv")
test = pd.read_csv("location_test.csv")

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [16]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [18]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

# Pre-processing

In [20]:
# not necessary

# Exploratory data analysis

In [21]:
X_train.isnull().sum()

1      0
2      0
3      0
4      0
5      0
      ..
442    0
443    0
444    0
445    0
446    0
Length: 446, dtype: int64

# Model selection

In [22]:
# Grid Search

cv = 10         # number of folds
verbose = 1     # information shown during training

## Stochastic Gradient Descent

In [23]:
parameters = {
    "loss":["hinge", "modified_huber", "log"],
    "penalty":["l1", "l2", "elasticnet"]}
model = GridSearchCV(SGDClassifier(), parameters, cv=cv, verbose=verbose, scoring="f1_weighted")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_loss", "param_penalty", "mean_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


,param_loss,param_penalty,mean_test_score
8,log,elasticnet,0.558852
7,log,l2,0.557172
2,hinge,elasticnet,0.554925
4,modified_huber,l2,0.554600
1,hinge,l2,0.553121
6,log,l1,0.551992
0,hinge,l1,0.550144
5,modified_huber,elasticnet,0.548386
3,modified_huber,l1,0.544675


# Final model

In [24]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

SGDClassifier(loss='log', penalty='elasticnet')

In [25]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [26]:
predictions.to_csv("submission.csv", index=False)